## **Розширені алгоритми прогнозування за допомогою нейронних мереж на прикладах медичних даних**

## Мета роботи

 Ця лабораторна робота присвячена прогнозуванню динаміки поширення COVID-19 у світі за допомогою нейронних мереж різних структур. Мета роботи навчитисяч робити прогнози на основі лінійної регресії, зворотного розповсюдження та нейронних мереж довгої короткочасної пам’яті (long short-term memory LSTM) 

## Виконання

Сьогодні існує багато відкритих даних про поширення COVID-19 у світі. Однак представлено небагато інструментів для прогнозування цих процесів. Ця лабораторна робота покаже, як можна завантажувати дані з відкритих джерел, виконувати попередній аналіз даних, перетворювати та очищати дані, виконувати кореляційний та затримковий аналіз.

Далі будуть розглянуті три різні математичні моделі для розрахунку прогнозування.

Для цього буде продемонстровано поділ набору даних на навчальні та тестові вибірк. Буде продемонстровно як нормалізувати дані та провести попередній аналіз. Буде показано, як побудувати моделі та набори даних для використання двох різних нейронних мереж. Наступним кроком є побудова прогнозу та порівняння точності та адекватності отриманих моделей.

## Матеріали та методи

У цій лабораторній роботі буде вивчено основні методи прогнозування часових рядів. Лабораторна робота складається з таких етапів:
* Завантаження та попередній аналіз даних
* Лінійна регресія
* Метод зворотного поширення помилки (Back Propagation NN)
* Довгої короткочасної пам’яті- LSTM

На першому етапі буде показано, як завантажити дані та заздалегідь підготувати їх до аналізу:
* завантажити дані
* змінити типи даних стовпців
* фільтрація рядків
* усунення відсутніх даних
* перетворення набору даних
* нормалізація даних

Під час наступних кроків буде продемонстрованмо три різні моделі прогнозування часових рядів.

Статистичні дані отримані з https://ourworldindata.org/coronavirus на основі Creative Commons BY license.

## Середовище виконання
* [Python](https://www.python.org)
* [Pandas](https://pandas.pydata.org)
* Statistics
* [NumPy](https://numpy.org)
* [Matplotlib](https://matplotlib.org)
* [Keras](https://keras.io)
* [Scikit-Learn](https://scikit-learn.org)

## Отримані навики


Після виконання даної лабораторної роботи отримуються навики:

* Завантаження набору даних з файлів *.csv
* Автоматичне змінення даних в наборі даних
* Перетворення таблиць
* Візуалізація даних за допомогою  pandas
* Створення прогнозних моделей на основі нейронних мереж


## Завантаження бібліотек

Необхідне встановлення додаткових бібліотек для виконання лабораторної роботи.

In [ ]:
conda install -c conda-forge keras --yes

In [ ]:
conda install -c conda-forge tensorflow --yes

## Завантаження та попередній аналіз даних

### Завантаження даних

Перед початком необхідно імпортувати додаткові бібліотеки.

In [1]:
import pandas as pd
import numpy as np

Будемо використовувати той самий набір даних, що і в попередній лабораторній роботі. 
Тому повторюємо  попередню обробку даних , як це було раніше у лабораторній роботі 2. 
Наступним кроком є завантаження файлу даних із[open repository produced by Our World in Data under the Creative Commons BY license](https://ourworldindata.org/coronavirus)
 з допомогою **[read_csv()](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)**.

In [ ]:
covid_word = pd.read_csv('owid-covid-data.csv')
covid_word

Давайте вивчимо наш набір даних. Він складається з 86202 рядків × 59 стовпців. 
Перші 3 стовпці містять географічну інформацію. Графа 4 - дата вимірювання. 
Ще 55 - дані про COVID -19. Також у наборі даних спостерігаються деякі відсутні дані. 
Ми повинні бути впевнені, що Python правильно розпізнав типи даних. Для цього ми повинні використовувати **[pandas.info()] 
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html?highlight=info#pandas.DataFrame.info)**.**[pandas.info()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html?highlight=info#pandas.DataFrame.info)**.

In [ ]:
covid_word.info()

Бачимо, 54 стовпці даних COVID-19 були розпізнані правильно (float64).
Перші 4 стовпці та одиниці тесту були визнані об’єктами. Давайте дослідимо їх:

In [ ]:
fields = ['iso_code', 'continent', 'location', 'tests_units']
covid_word[fields]

Необхідно відобразити інформацію про поле дата.

<details><summary>Click <b>here</b> for the solution</summary> <code>
covid_word['date']
</code></details>

### Зміна типів стовпців

As you can see, the columns: 'iso_code', 'continent', 'location', 'tests_units' have many repetitions and should be assigned to categorical fieldsБачимо, що колонки: 'iso_code', 'continent', 'location', 'tests_units'мають багато повторів і повинні бути віднесені до категорійних полів 
**([pandas.astype()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html?highlight=astype#pandas.DataFrame.astype))**.
Поле "дата" слід перетворити на тип DateTime   **([pandas.to_datetime()](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html))**. Для перегляду результатів можна використати наступний код  **.

In [ ]:
fields = ['iso_code', 'continent', 'location', 'tests_units']
covid_word[fields] =covid_word[fields].astype('category')
covid_word.loc[:, 'date'] = pd.to_datetime(covid_word['date'])
covid_word[fields].describe()

### Фільтрація рядків

Бачимо, що набір даних містить інформацію про 6 континентах та 219 країнах. Поле "test_units" складається з 4 категорій. Щоб показати список країн, треба використати**[pandas.Series.cat.categories](https://pandas.pydata.org/docs/reference/api/pandas.Series.cat.categories.html)**.

In [ ]:
covid_word['location'].cat.categories

Let's investigate the dynamics of new cases of COVID-19 for a separate country. We will use Ukraine for saving models and further forecast. You can use your country. Let's use a pandas filter to do this.Дослідимо динаміку нових випадків COVID-19 для окремої країни. Будемо використовувати Україну для збереження моделей та подальшого прогнозу. 
Вибір країн здійснюємо згідно індивідуального завдання. 
Використовуємо для цього фільтр pandas.


In [ ]:
covid_word.index = covid_word['date']
##YOUR CODE GOES HERE##
<details><summary>Click <b>here</b> for the solution</summary> <code>
c_covid = covid_word[covid_word['location'] == "Ukraine"]
</code></details>
c_covid

Виберемо поля 'new_cases', 'new_cases_smoothed' для прогнозування. Перш за все, необхідно візуалізувати ці дані.

In [ ]:
<details><summary>Click <b>here</b> for the solution</summary> 
<code>
import matplotlib.pyplot as plt
fields = ['new_cases', 'new_cases_smoothed']
c_covid[fields].plot()
plt.show()
</code>
</details>

###  Видалення пропущених даних

Бачимо, у графіку нових випадків є великі коливання. Спробуємо скласти прогноз для цих хвиль. Перш за все, ми повинні видалити відсутні дані за допомогою [*pandas.DataFrame.dropna()*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [ ]:
c_covid = c_covid[fields].dropna()
c_covid

### Перетворення даних

Якщо ми хочемо зробити прогноз часових рядів, єдине припущення, яке можна зробити - дані на сьогоднішній день залежать від значень попередніх днів. Щоб перевірити чи є залежності, необхідно провести кореляційний аналіз між ними. Для цього потрібно зробити:
1. дублювання часового ряду даних та переміщення даних вертикально вниз протягом певної кількості днів затримки (lag)
2. видалення відсутніх даних на початку та в кінці (вони формуються шляхом вертикального зсуву  (**[pandas.DataFrame.shift()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html)**)
3. обчислення коефіцієнта кореляції між отриманими рядами.

Оскільки цю операцію слід виконувати для різних значень затримки (lag), зручно створити окрему функцію:


In [ ]:
def lag_correlation_ts(y, x, lag):
    """
    Lag correlation for 2 DateSeries
    :param y: fixed
    :param x: shifted
    :param lag: lag for shifting
    :return: DataFrame of lags correlation coefficients
    """
    r = [0] * (lag + 1)
    y = y.copy()
    x = x.copy()
    y.name = "y"
    x.name = "x"

    for i in range(0, lag + 1):
        ds = y.copy().to_frame()
        ds = ds.join(x.shift(i), how='outer')
        r[i] = ds.corr().values[0][1]
    r = pd.DataFrame(r)
    r.index.names = ['Lag']
    r.columns = ['Correlation']
    return r

Cтворимо цільовий (target) набір даних.

In [ ]:
y_dataset = c_covid['new_cases']
y_dataset

Протестуємо 30-денну затримку.

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format
l = pd.DataFrame(lag_correlation_ts(y_dataset, y_dataset, 30)) #For time series we should use y_Dataset like input and output
print(l)
l.plot()

Бачимо, що на ділянці затримки спостерігаються деякі хвилі. З таблиці видно, що на кожен сьомий день припадає пік. (Максимальний час затримки: 7, 14, 21 тощо). Це пов'язано з тижневим циклом.
Будь-яку модель прогнозу можна показати у вигляді чорного ящика типу введення-цілі. Цільовими мають бути дані вихідного часового ряду, а вхідними - значення за попередні дні. (Див. рис.1)
Щоб автоматизувати цей процес, давайте зробимо універсальну функцію перетворення часових рядів для створення цього набору даних.

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-health-care-advanced-prognostication-using-by-neural-networks/FM.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

Рис.1 Структура моделі прогнозування


Щоб автоматизувати цей процес, треба зробити універсальну функцію перетворення часових рядів для створення цього набору даних.

In [ ]:
def series_to_supervised(in_data, tar_data, n_in=1, dropnan=True, target_dep=False):
    """
    Transformation into a training sample taking into account the lag
     : param in_data: Input fields
     : param tar_data: Output field (single)
     : param n_in: Lag shift
     : param dropnan: Do destroy empty lines
     : param target_dep: Whether to take into account the lag of the input field. If taken into account, the input will start with a lag 1
     : return: Training sample. The last field is the source
    """

    n_vars = in_data.shape[1]
    cols, names = list(), list()

    if target_dep:
        i_start = 1
    else:
        i_start = 0
    for i in range(i_start, n_in + 1):
        cols.append(in_data.shift(i))
        names += [('%s(t-%d)' % (in_data.columns[j], i)) for j in range(n_vars)]

    if target_dep:
        for i in range(n_in, -1, -1):
            cols.append(tar_data.shift(i))
            names += [('%s(t-%d)' % (tar_data.name, i))]
    else:
        # put it all together
        cols.append(tar_data)
        names.append(tar_data.name)
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    return agg

Як згадувалося вище, при прогнозуванні часових рядів поля введення та виведення є однаковими, лише зміщуються на затримку (lag).

In [ ]:
dataset = series_to_supervised(pd.DataFrame(y_dataset), y_dataset, 14)
dataset

Отже перший і останній стовпці містять однакові цільові дані. Тепер треба створити набори даних вхідних (**X**) та вихідних (**Y**) даних для моделей прогнозування.

In [ ]:
col = dataset.columns
X, Y = dataset[col[1:-1]], dataset[col[-1]]
print("Input: ", X.columns)
print("Target:", ##YOUR CODE GOES HERE## Y.name)

In [ ]:
### Нормування даних

Після цього ми повинні нормалізувати всі дані. Для цього слід використовувати модуль  [**sklearn.preprocessing.MinMaxScaler**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html). 
Це дозволяє нам нормалізувати   [**fit_transform()**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.fit_transform) легко конвертувати всі дані назад: [**fit_transform()**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.inverse_transform).


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

scaled_x = scaler_x.fit_transform(X)
scaled_y = scaler_y.fit_transform(Y.values.reshape(-1, 1))

After that, we are going to form training and test DataSets using [**sklearn.model_selection.train_test_split()**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). We will make them at the ratio of 70/30. Without shuffling. It means, that test samples are located in the end of **X** and **Y** DataSets.

Input normalized DataSets: **X_train, X_test**

Target normalized DataSets: **y_train, y_test**
Після цього ми збираємося сформувати навчальні та перевірити набори даних за допомогою [**sklearn.model_selection.train_test_split()**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
Ми зробимо їх у співвідношенні 70/30. Без перемішування. Це означає, що тестові зразки розташовані в кінці наборів даних X і Y.

Введення нормалізованих наборів даних: **X_train, X_test**

Цільові нормалізовані набори даних: **y_train, y_test**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_x, scaled_y, test_size=0.3, shuffle=False)

Зараз усі дані нормалізовані. Однак, для порівняння з результатами, нам потрібні дані реального масштабу навчального та тестового набору даних::

In [ ]:
res_train = scaler_y.inverse_transform(y_train).flatten()
res_test = scaler_y.inverse_transform(y_test).flatten()

Цільові набори даних реального масштабу: **res_train, res_test**.

# Лінійна регресія

Перш за все, треба створити модель. Ми перевіримо три типи моделей. Лінійна регресія, багатошарова нейронна мережа зі зворотним поширенням помилки та нейромережа з довгою короткостроковою пам'яттю. Створимо a [**LinearRegression()**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html): 

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

Після цього наша модель повинна бути тренована на навчальному наборі даних. Незалежно від типу моделі, для цього використовується функція fit ().

In [ ]:
regressor.fit(X_train, y_train)

Тоді ми можемо перевірити його на тестовому наборі даних та використовувати для прогнозування. 

In [ ]:
y_pred_test_ln = regressor.predict(X_test)
y_pred_test_ln = scaler_y.inverse_transform(y_pred_test_ln).flatten()

Аналізуємо точність моделі, використовуючи **[sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)**.

In [ ]:
from sklearn import metrics
print("Correlation train", regressor.score(X_train, y_train))
print("Correlation test", regressor.score(X_test, y_test))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_ln))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_ln))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_ln)))

# Метод зворотного поширення помилки (Back Propagation NN)

Сучасний підхід до встановлення складних функціональних залежностей полягає у використанні нейронних мереж. Класична нейронна мережа - це багатошарова нейронна мережа із зворотним поширенням. [**multilayer neural network with back propagation**](https://en.wikipedia.org/wiki/Backpropagation).

Для цього використаємо [**keras**](https://keras.io) фреймворк.
Перш за все, ми повинні створити модель нейронної мережі як окрему функцію.

Нейронна мережа - це послідовність шарів. Функція Sequential ()  [**Sequential()**](https://keras.io/guides/sequential_model/) використовується для створення мережі.

Вихідний шар буде складатися з одного нейрона, оскільки у нас на виході є лише одне значення. [**keras.layers.Dense()**](https://keras.io/api/layers/core_layers/dense/).

Щоб уникнути проблем з перенавчанням, ми будемо використовувати додаткові шари[**keras.layers.Dropout()**](https://keras.io/api/layers/regularization_layers/dropout/).

Модель необхідно зкомпілювати для підгонки та прогнозування:
 [**keras.Model.compile()**](https://keras.io/api/models/model_training_apis/).
Сучасний підхід до визначення складних функціональних залежностей полягає у використанні нейронних мереж. Класична нейронна мережа - це багатошарова нейронна мережа із зворотним поширенням.

Для цього ми будемо використовувати фреймворк keras. Перш за все, ми повинні створити модель нейронної мережі як окрему функцію.

Нейронна мережа - це послідовність шарів. Функція Sequential () використовується для створення мережі.

Створимо мережу, яка складається з 2 прихованих шарів. Кожен з яких складається з 100 нейронів. keras.layers.Dense ().

Щоб уникнути проблем з перепідготовкою, ми будемо використовувати додаткові шари keras.layers.Dropout ().

Вихідний шар буде складатися з одного нейрона, оскільки у нас на виході є лише одне значення.

Модель слід зібрати для підгонки та прогнозування: keras.Model.compile ().

In [ ]:
def BP_model(X):
    """
    Multilayer neural network with back propagation.
    :param X: Input DataSet
    :return: keras NN model
    """
    # create model
    model = Sequential() 
    model.add(Dense(100, input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

Після того, як функція моделі буде побудована, необхідно безпосередньо створити нейронну мережу та вказати параметри навчання:   [**keras.wrappers.scikit_learn.KerasRegressor()**](https://keras.io/zh/scikit-learn-api/). Також слід визначити кількість епох припасування та розмір навчальної вибірки: [**epoch and batch size**](https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/).

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
epochs = 1000

batch_size=int(y_train.shape[0]*.1)

estimator = KerasRegressor(build_fn=BP_model, X=X_train, epochs=epochs, batch_size=batch_size, verbose=1)

Now, let’s train our model for  epochs.
It should be noted that the fitting process is very slow. Therefore, we saved our fitted model to a file.
To save time, we will upload the fitted model.
If you like, you can leave the parameter  to refit the model.
If you like, you can leave the parameter  to resave the model.Тепер давайте навчимо нашу модель **1000** епох. Слід зазначити, що процес підгонки відбувається дуже повільно. Тому труба зберегли таку модель у файл. Щоб заощадити час,  завантажимо підігнану модель. Якщо потрібно,  можна залишити параметр **fitting on True** рівним True, щоб оновити модель. Якщо потрібно,  можна залишити параметр **fitting_save on True**, щоб зберегти модель.

In [ ]:
fitting = True
fitting_save = True

import pickle

if fitting:
    history=estimator.fit(X_train,y_train, validation_data=(X_test,y_test)) # Fitting model
    if fitting_save:
        # Save model
        estimator.model.save('BP_saved_model.h5')
        print("Saved model to disk")
        with open('history.pickle', 'wb') as f:
            pickle.dump(history.history, f)
# load model 
from keras.models import load_model

# Instantiate the model as you please (we are not going to use this)
estimator = KerasRegressor(build_fn=BP_model, X=X_train, epochs=epochs, batch_size=batch_size, verbose=1)
# This is where you load the actual saved model into a new variable.
estimator.model = load_model('BP_saved_model.h5')    
with open('history.pickle', 'rb') as f:
    history = pickle.load(f)
print("Loaded model from disk")

Покажемо динаміку втрат та валідації [**loss and validation loss dynamics**](https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/).

In [ ]:
plt.figure()
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='test')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.plot(history.history['acc'], label='acc')
#plt.plot(history.history['val_acc'], label='acc test')
plt.legend()
plt.show()

Отже нейромережа добре підігнана, і не спостерігається її надмірної адаптації. Давайте обчислимо прогноз навчальної (**res_train_ANN**) і тестової  (**res_test_ANN**) вибірки.


Давайте обчислимо прогноз і зробимо зворотну нормалізацію до реального масштабу.

In [ ]:
res_tr=estimator.predict(X_train)
res_ts=estimator.predict(X_test)
res_train_ANN=scaler_y.inverse_transform(res_tr.reshape(-1, 1)).flatten()
res_test_ANN=scaler_y.inverse_transform(res_ts.reshape(-1, 1)).flatten()

Давайте порівняємо точність лінійної регресії та нейронної мережі.

In [ ]:

print("Correlation train", np.corrcoef(res_train, res_train_ANN)[0,1])
print("Correlation train", np.corrcoef(res_test, res_test_ANN)[0,1])
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, res_test_ANN))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, res_test_ANN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, res_test_ANN)))

 Можна бачити, що отримано дещо кращі результати для нейронної мережі, ніж для лінійної регресії.

# Метод довгої короткочасної пам’яті Long Short-Term Memory - LSTM

 На відміну від стандартних нейронних мереж, що передають інформацію, [** LSTM **] (https://en.wikipedia.org/wiki/Long_short-term_memory) має зв'язки зворотного зв'язку. Він може обробляти не тільки окремі точки даних, а й цілі послідовності даних (наприклад, мовлення, відео чи часові ряди).

У разі часових рядів нейронна мережа має один вхід і один вихід. Однак на вході повинен бути вектор значень часових рядів за попередній період часу.

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/data-science-in-health-care-advanced-prognostication-using-by-neural-networks/LSTM.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

Для цього нам слід перетворити вхідні набори даних у 3D -форму.

In [ ]:
train_x_LSTM = X_train.reshape((X_train.shape[0], 1, 14))
test_x_LSTM = X_test.reshape((X_test.shape[0], 1, 14))

Давайте створимо нейронну мережу [**LSTM**](https://keras.io/api/layers/recurrent_layers/lstm/) що складається з одного шару LSTM та одного шару BP, як у попередньому випадку.
As you can see, in this case our NN will consist of 7 LSTM and 7 BP neurons only. LSTM, що складається з одного шару LSTM та одного шару BP, як у попередньому випадку. Як бачите, у цьому випадку наша NN складатиметься лише з 7 нейронів LSTM і 7 нейронів шару BP .

In [ ]:
from keras.layers import LSTM

batch_size=int(y_train.shape[0]*.1)
model = Sequential()
model.add(LSTM(7, input_shape=(train_x_LSTM.shape[1], train_x_LSTM.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(7, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1])) #activation='sigmoid'
model.compile(loss='mean_squared_error', optimizer='adam')
#model.compile(loss='mae', optimizer='adam')


Усі наступні етапи навчання, зберігання, читання та прогнозування подібні до тих, які ми використовували з попередньою нейронною мережею. Можна бачити, що всього 400 епох достатньо для LSTM.

In [ ]:
fitting = True
fitting_save = True
epochs = 400

if fitting:
    history = model.fit(train_x_LSTM, y_train, epochs=epochs, batch_size=batch_size, validation_data=(test_x_LSTM, y_test), verbose=1, shuffle=False)    
    if fitting_save:
    # serialize model to JSON
        model_json = model.to_json()
        with open("LSTM_model.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model.save_weights("LSTM_model.h5")
        print("Saved model to disk")
        with open('history_LSTM.pickle', 'wb') as f:
            pickle.dump(history.history, f)
# load model  
from keras.models import model_from_json
json_file = open('LSTM_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("LSTM_model.h5")        
with open('history_LSTM.pickle', 'rb') as f:
    history = pickle.load(f)
print("Loaded model from disk")


Давайте побудуємо динаміку втрат та втрат значень, як у попередньому випадку. 

In [ ]:
##YOUR CODE GOES HERE## 
# plot history
plt.figure()
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='test')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend()
plt.show()

Let's calculate our forecast.

In [ ]:
 
res_tr_LSTM = model.predict(train_x_LSTM)
res_ts_LSTM = model.predict(test_x_LSTM)
res_train_LSTM=scaler_y.inverse_transform(res_tr_LSTM).flatten()
res_test_LSTM=scaler_y.inverse_transform(res_ts_LSTM).flatten()

And accuracy:

In [ ]:

print("Correlation train", np.corrcoef(res_train, res_train_LSTM)[0,1])
print("Correlation train", np.corrcoef(res_test, res_test_LSTM)[0,1])
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, res_test_LSTM))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, res_test_LSTM))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, res_test_LSTM)))

As you can see, the forecast results of the test data set are much better than ones of the previous models. Let's visualize these 3 results:

In [ ]:
res_pred_test_ln = pd.Series(y_pred_test_ln, name = 'Predicted test Linear Model')
res_pred_test_ANN = pd.Series(res_test_ANN, name = 'Predicted test ANN')
res_pred_test_LSTM = pd.Series(res_test_LSTM, name = 'Predicted test LSTM')

df_2 = pd.DataFrame({'Actual test': res_test, 'Linear Model': res_pred_test_ln, 'ANN Model': res_pred_test_ANN,  'LSTM Model': res_pred_test_LSTM,})
df_2.index = dataset.index[len(dataset)-len(res_test):]
df_2.plot()
plt.show()

Бачимо,що модель LSTM дає ідеальний прогноз. Лінійна регресія - найшвидша модель прогнозування.

## Висновки

У цій лабораторній роботі розглянуто три типи моделей прогнозування. Реалізовано перетворення наборів даних для моделей введення-виведення. Здійснено нормалізація та обернена нормалізація для даних реального масштабу. Крім того,  продемонстровано, як розділити набори даних на навчальні та тестові. Показано, як побудувати моделі прогнозування часових рядів, використовуючи лагові перетворення. Розглянуто як встановлювати, зберігати та завантажувати різні типи нейронних мереж.

# Індивідуальне завдання
Реалізувати прогнозування за трьома моделями для групи країн з індивідуального завдання на основі даних https://ourworldindata.org/coronavirus. Результати порівняти.


1. Реалізувати прогнозування поширення COVID-19 у групі скандинавських країн. 

2. Реалізувати прогнозування поширення COVID-19 у групі  країн Benelux. 
3. Реалізувати прогнозування поширення COVID-19 у групі  країн Європейського Союзу.

4. Реалізувати прогнозування поширення COVID-19 у групі  країн, що повністю розташовані на Аравійському півострові. 

5. Реалізувати прогнозування поширення COVID-19 у групі  країн Південна Корея, Гонконг, Сінгапур, Тайвань. 
6. Реалізувати прогнозування поширення COVID-19 у групі  країн Монголія, Китай і Вєтнам. 
7. Реалізувати прогнозування поширення COVID-19 у групі  Африканських країн, що повністю лежать у північній півкулі. 

8. Реалізувати прогнозування поширення COVID-19 у групі  Африканських країн, що повністю лежать у південній півкулі. 

9. Реалізувати прогнозування поширення COVID-19 у групі країн Індія, Непал, Пакистан. 

10. Реалізувати прогнозування поширення COVID-19 у групі країн Південної Америки.

11. Реалізувати прогнозування поширення COVID-19 у групі прибалтійських  країн. 

12. Реалізувати прогнозування поширення COVID-19 у групі  балканських країн. 

13. Реалізувати прогнозування поширення COVID-19 у групі країн Польща, Угорщина, Чехія, Словаччина, Словенія, Хорватія. 
14. Реалізувати прогнозування поширення COVID-19 у групі країн Болгарія, Румунія, Україна, Албанія, Македонія.

15. Реалізувати прогнозування поширення COVID-19 у групі країн Канада, США, Мексика. 


​

## Автор

[Yaroslav Vyklyuk, prof., PhD., DrSc](http://vyklyuk.bukuniver.edu.ua/en/)